In [1]:
import pandas as pd
!conda install -c conda lxml
!pip install et_xmlfile
!pip install bs4
!pip install html5lib
!pip install lxml
import bs4.builder._lxml
from lxml import etree
import requests
from bs4 import BeautifulSoup

Solving environment: ...working... done

# All requested packages already installed.



DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


For scraping a table from the webpage

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content,features="lxml")
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
neighborhood=pd.DataFrame(df[0])

In [3]:
neighborhood.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Ignore cells with a borough that is Not assigned

In [3]:
neighborhood = neighborhood[neighborhood.Borough != 'Not assigned']

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [4]:
neighborhood['Neighborhood']=neighborhood['Neighborhood'].replace('Not assigned', neighborhood['Borough'])

In [5]:
neighborhood.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
neighborhood.shape

(103, 3)

In [7]:
import pandas as pd
geo_df = pd.read_csv('http://cocl.us/Geospatial_data', index_col= 0)

In [8]:
geo_df.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


get the latitude and the longitude coordinates of each neighborhood

In [9]:
df = neighborhood.merge(geo_df,left_on='Postal Code',right_on='Postal Code')

In [10]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Work with only boroughs that contain the word Toronto

In [11]:
df = df[df['Borough'].astype(str).str.contains('Toronto')]

In [12]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [14]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Solving environment: ...working... done

# All requested packages already installed.



SyntaxError: invalid syntax (algolia.py, line 25)

Try to explore the first borough in the dataframe (Downtown Toronto, Regent Park, Harbourfront)

In [15]:
df.loc[2, ['Borough', 'Neighborhood']]

Borough                  Downtown Toronto
Neighborhood    Regent Park, Harbourfront
Name: 2, dtype: object

In [16]:
CLIENT_ID = 'RXCSBILFIB02UXT4ZWX4P5Z2N5LS1THYLU3XREITE20M40RN' # your Foursquare ID: RXCSBILFIB02UXT4ZWX4P5Z2N5LS1THYLU3XREITE20M40RN
CLIENT_SECRET = 'OUSEO2PMUTMVK5KFEPOXRLHDGPTUGLDSXWV5V2C3A25Z5P4X' # your Foursquare Secret: OUSEO2PMUTMVK5KFEPOXRLHDGPTUGLDSXWV5V2C3A25Z5P4X
VERSION = '20180605'

Now, let's get the top 2 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL.

In [17]:
radius = 500
LIMIT = 100
Search_query = 'Borough'
latitude = df.loc[2, 'Latitude']
longitude = df.loc[2,'Longitude']
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format (CLIENT_ID, CLIENT_SECRET,VERSION,latitude,longitude,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RXCSBILFIB02UXT4ZWX4P5Z2N5LS1THYLU3XREITE20M40RN&client_secret=OUSEO2PMUTMVK5KFEPOXRLHDGPTUGLDSXWV5V2C3A25Z5P4X&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

Send the GET request and examine the resutls

In [18]:
results = requests.get(url).json()
results

{u'meta': {u'code': 200, u'requestId': u'5f02f4d328796f3c9cf076e6'},
 u'response': {u'groups': [{u'items': [{u'reasons': {u'count': 0,
       u'items': [{u'reasonName': u'globalInteractionReason',
         u'summary': u'This spot is popular',
         u'type': u'general'}]},
      u'referralId': u'e-0-54ea41ad498e9a11e9e13308-0',
      u'venue': {u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          u'suffix': u'.png'},
         u'id': u'4bf58dd8d48988d16a941735',
         u'name': u'Bakery',
         u'pluralName': u'Bakeries',
         u'primary': True,
         u'shortName': u'Bakery'}],
       u'id': u'54ea41ad498e9a11e9e13308',
       u'location': {u'address': u'362 King St E',
        u'cc': u'CA',
        u'city': u'Toronto',
        u'country': u'Canada',
        u'crossStreet': u'Trinity St',
        u'distance': 143,
        u'formattedAddress': [u'362 King St E (Trinity St)',
         u'Toronto ON M5A 1K9',
         u'Canada'

let's borrow the get_category_type function from the Foursquare lab.

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Dominion Pub and Kitchen,Pub,43.656919,-79.358967


And how many venues were returned by Foursquare?

In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*

In [23]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

#### Let's check the size of the resulting dataframe

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(1613, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


#### Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


## 3. Analyze Each Neighborhood

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [27]:
toronto_onehot.shape

(1613, 234)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.017241,0.0000,0.017241,0.034483,0.000000,0.000000,0.00,0.017241,0.017241,0.000000,0.034483,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.034483,0.000000,0.000000,0.0000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.017241,0.051724,0.086207,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.017241,0.000000,0.017241,0.00,0.000000,0.00,0.000000,0.000,0.017241,0.000000,0.000000,0.000000,0.000000

In [29]:
toronto_grouped.shape

(39, 234)

#### Let's print each neighborhood along with the top 10 most common venues

In [30]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2         Cheese Shop  0.03
3              Bakery  0.03
4            Beer Bar  0.03
5          Restaurant  0.03
6                Café  0.03
7  Seafood Restaurant  0.03
8   French Restaurant  0.02
9              Bistro  0.02


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.14
1          Breakfast Spot  0.09
2             Coffee Shop  0.09
3               Pet Store  0.05
4            Intersection  0.05
5           Burrito Place  0.05
6       Convenience Store  0.05
7                 Stadium  0.05
8                     Bar  0.05
9  Furniture / Home Store  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0            Restaurant  0.06
1         Auto Workshop  0.06
2               Brewery  0.06
3  Fast Food Restaurant  0.06
4            Smoke S

                  venue  freq
0        Clothing Store  0.20
1           Coffee Shop  0.10
2           Yoga Studio  0.05
3   Sporting Goods Shop  0.05
4    Italian Restaurant  0.05
5  Fast Food Restaurant  0.05
6    Mexican Restaurant  0.05
7                 Diner  0.05
8                  Park  0.05
9    Chinese Restaurant  0.05


----Parkdale, Roncesvalles----
                         venue  freq
0               Breakfast Spot  0.13
1                    Gift Shop  0.13
2                          Bar  0.07
3                Movie Theater  0.07
4                      Dog Run  0.07
5                 Dessert Shop  0.07
6           Italian Restaurant  0.07
7                   Restaurant  0.07
8  Eastern European Restaurant  0.07
9                         Bank  0.07


----Queen's Park, Ontario Provincial Government----
               venue  freq
0        Coffee Shop  0.22
1              Diner  0.06
2   Sushi Restaurant  0.06
3        Yoga Studio  0.03
4  College Cafeteria  0.03
5      Smoothi

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Seafood Restaurant,Bakery,Restaurant,Beer Bar,Lounge,Liquor Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Performing Arts Venue,Pet Store,Convenience Store,Climbing Gym,Restaurant,Burrito Place
2,"Business reply mail Processing Centre, South C...",Park,Auto Workshop,Comic Shop,Pizza Place,Butcher,Restaurant,Burrito Place,Brewery,Light Rail Station,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
4,Central Bay Street,Coffee Shop,Sandwich Place,Japanese Restaurant,Italian Restaurant,Café,Department Store,Thai Restaurant,Bubble Tea Shop,Burger Joint,Bar


Find out which neighbourhood has the 1st most common venue is coffee shop. These are potential neighbourhoods.

In [33]:
potential_neighborhoods=neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['1st Most Common Venue']=='Coffee Shop']
potential_neighborhoods['Neighborhood']

0                                           Berczy Park
4                                    Central Bay Street
6                                  Church and Wellesley
7                        Commerce Court, Victoria Hotel
11               First Canadian Place, Underground city
14    Harbourfront East, Union Station, Toronto Islands
23          Queen's Park, Ontario Provincial Government
24                            Regent Park, Harbourfront
25                             Richmond, Adelaide, King
29                                       St. James Town
30                          St. James Town, Cabbagetown
31                                       Stn A PO Boxes
37             Toronto Dominion Centre, Design Exchange
Name: Neighborhood, dtype: object

Let create a new dataframe only including the potential neighbourhoods

In [34]:
df_potential_neighborhoods=toronto_grouped.loc[toronto_grouped['Neighborhood'].isin(potential_neighborhoods['Neighborhood'])]
df_potential_neighborhoods.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.017241,0.000000,0.000000,0.00,0.0,0.017241,0.0,0.017241,0.034483,0.0,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.00,0.017241,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.034483,0.0,0.0,0.0,0.000000,0.034483,0.0,0.0,0.0,0.0,0.017241,0.051724,0.086207,0.0,0.0,0.0,0.0,0.0,0.00,0.017241,0.000000,0.017241,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.00,0.017241,0.000000,0.017241,0.000000,0.000000,0.000000,0.0,0.000000,0.017241,0.0,0.0,0.0,0.000000,0.017241,0.000000,0.0,0.0,0.017241,0.0,0.0,0.000000,0.00,0.00,0.017241,0.017241,0.0,0.0,0.0

Add a new column 'critical value', which substract the frequency of the coffee shop to the frequency of the cafe

In [35]:
df_potential_neighborhoods['critical_value'] = df_potential_neighborhoods['Coffee Shop'] - df_potential_neighborhoods.iloc[:,44]

C:\Users\mtv1g12\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
df_potential_neighborhoods.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,critical_value
0,Berczy Park,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.017241,0.000000,0.000000,0.00,0.0,0.017241,0.0,0.017241,0.034483,0.0,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.00,0.017241,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.034483,0.0,0.0,0.0,0.000000,0.034483,0.0,0.0,0.0,0.0,0.017241,0.051724,0.086207,0.0,0.0,0.0,0.0,0.0,0.00,0.017241,0.000000,0.017241,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.00,0.017241,0.000000,0.017241,0.000000,0.000000,0.000000,0.0,0.000000,0.017241,0.0,0.0,0.0,0.000000,0.017241,0.000000,0.0,0.0,0.017241,0.0,0.0,0.000000,0.00,0.00,0.017241,0.017

Sort the dataframe from biggest 'critical value' to smallest one to identify the best neighbourhood.

In [37]:
df_potential_neighborhoods = df_potential_neighborhoods.sort_values(by='critical_value', ascending=False)
df_potential_neighborhoods.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,critical_value
23,"Queen's Park, Ontario Provincial Government",0.027778,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.027778,0.000000,0.0,0.0,0.0,0.0,0.000000,0.027778,0.027778,0.00,0.00,0.0,0.0,0.027778,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.027778,0.0,0.0,0.027778,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.222222,0.0,0.027778,0.027778,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.00,0.000000,0.0,0.027778,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.055556,0.027778,0.027778,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0

# After choosing the neighbourhood of Queen's Park, Ontario Provincial Government, find the potential specific location

In [45]:
df_potential_neighborhoods.iloc[0,0]

"Queen's Park, Ontario Provincial Government"

# Obtain latitude and longitude of all coffee shops in the chosen neighbourhood

In [49]:
df2=toronto_venues.loc[toronto_venues['Neighborhood']== df_potential_neighborhoods.iloc[0,0]]
df2

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
46,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Queen's Park,43.663946,-79.392180,Park
47,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Mercatto,43.660391,-79.387664,Italian Restaurant
48,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Neo Coffee Bar,43.660140,-79.385870,Coffee Shop
49,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Sushi Box,43.662960,-79.386580,Sushi Restaurant
50,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Central YMCA,43.663083,-79.385025,Distribution Center
51,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Coffee Public,43.660763,-79.386184,Coffee Shop
52,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,T-Swirl Crepe,43.663452,-79.384125,Creperie
53,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Bar Volo,43.665462,-79.385692,Beer Bar
54,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tokyo Sushi,43.665885,-79.386977,Sushi Restaurant
55,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio


In [51]:
coffee_shop_location=df2.loc[df2['Venue Category']== 'Coffee Shop']
coffee_shop_location

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
48,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Neo Coffee Bar,43.660140,-79.385870,Coffee Shop
51,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Coffee Public,43.660763,-79.386184,Coffee Shop
65,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Starbucks,43.658204,-79.388998,Coffee Shop
70,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.661038,-79.393797,Coffee Shop
71,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Starbucks,43.660887,-79.393720,Coffee Shop
73,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.659415,-79.391221,Coffee Shop
76,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.658175,-79.390681,Coffee Shop
77,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.658906,-79.388696,Coffee Shop


There are eight coffee shops in the chosen neigbourhood. The idea is find which area nearby these shops (in a radius of 200 meters) has the highest density of venues involving coffee. This area can be the best location for opening a new coffe shop. 

# 1. Test for area nearby Neo Coffee Bar

In [80]:
latitude = coffee_shop_location.iloc[0,4]
longitude = coffee_shop_location.iloc[0,5]
search_query = 'Coffee'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results
venues = results['response']['venues']
Neo_Coffee_Bar = json_normalize(venues)
Neo_Coffee_Bar.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,5db70a0c306d6b000861ac9f,770 Bay Street Unit 3,CA,Toronto,Canada,NaN,0,"[770 Bay Street Unit 3, Toronto ON M5G 0A6, Ca...","[{u'lat': 43.66014, u'lng': -79.38587, u'label...",43.660140,-79.385870,NaN,M5G 0A6,ON,Neo Coffee Bar,v-1594034389
1,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,4b156e98f964a520cbac23e3,801 Bay St,CA,Toronto,Canada,at College St,70,"[801 Bay St (at College St), Toronto ON M5S 1Y...","[{u'lat': 43.6607135392, u'lng': -79.385491258...",43.660714,-79.385491,NaN,M5S 1Y9,ON,Timothy's World Coffee,v-1594034389
2,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,4bce5e21cc8cd13a7359c4cf,444 Yonge St,CA,Toronto,Canada,in College Park,104,"[444 Yonge St (in College Park), Toronto ON M5...","[{u'lat': 43.6604673968, u'lng': -79.384653568...",43.660467,-79.384654,NaN,M5B 2H4,ON,Timothy's World Coffee,v-1594034389
3,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,568bcf61498e3aaff5e351f8,800 Bay St,CA,Toronto,Canada,at College St,73,"[800 Bay St (at College St), Toronto ON M5S 3A...","[{u'lat': 43.6607629915, u'lng': -79.386184090...",43.660763,-79.386184,NaN,M5S 3A9,ON,Coffee Public,v-1594034389
4,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,56b2d97f498ebb2041ae6502,800 Bay St,CA,Toronto,Canada,NaN,115,"[800 Bay St, Toronto ON M5S 3A9, Canada]","[{u'lat': 43.6611504233, u'lng': -79.386198520...",43.661150,-79.386199,NaN,M5S 3A9,ON,Coffee Place,v-1594034389


In [81]:
Neo_Coffee_Bar.shape

(8, 18)

There are 8 venues relating coffee nearby Neo Coffee Bar. 

# 2. Test for area nearby Coffee Public

In [87]:
latitude_1 = coffee_shop_location.iloc[1,4]
longitude_1 = coffee_shop_location.iloc[1,5]
search_query = 'Coffee'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_1, longitude_1, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results
venues = results['response']['venues']
Coffee_Public = json_normalize(venues)
Coffee_Public.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,4b156e98f964a520cbac23e3,801 Bay St,CA,Toronto,Canada,at College St,56,"[801 Bay St (at College St), Toronto ON M5S 1Y...","[{u'lat': 43.6607135392, u'lng': -79.385491258...",43.660714,-79.385491,M5S 1Y9,ON,Timothy's World Coffee,v-1594035145
1,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,4bce5e21cc8cd13a7359c4cf,444 Yonge St,CA,Toronto,Canada,in College Park,127,"[444 Yonge St (in College Park), Toronto ON M5...","[{u'lat': 43.6604673968, u'lng': -79.384653568...",43.660467,-79.384654,M5B 2H4,ON,Timothy's World Coffee,v-1594035145
2,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,568bcf61498e3aaff5e351f8,800 Bay St,CA,Toronto,Canada,at College St,0,"[800 Bay St (at College St), Toronto ON M5S 3A...","[{u'lat': 43.6607629915, u'lng': -79.386184090...",43.660763,-79.386184,M5S 3A9,ON,Coffee Public,v-1594035145
3,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,5db70a0c306d6b000861ac9f,770 Bay Street Unit 3,CA,Toronto,Canada,NaN,73,"[770 Bay Street Unit 3, Toronto ON M5G 0A6, Ca...","[{u'lat': 43.66014, u'lng': -79.38587, u'label...",43.660140,-79.385870,M5G 0A6,ON,Neo Coffee Bar,v-1594035145
4,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,5d850196f516bc00089d2488,800 Bay St,CA,Toronto,Canada,NaN,11,"[800 Bay St, Toronto ON M5S 3A9, Canada]","[{u'lat': 43.660867, u'lng': -79.386206, u'lab...",43.660867,-79.386206,M5S 3A9,ON,800 Coffee Bar,v-1594035145


In [88]:
Coffee_Public.shape

(7, 17)

There are 7 venues relating coffee nearby Coffee Public. 

# 3. Test for area nearby Starbucks shop 1

In [91]:
latitude_2 = coffee_shop_location.iloc[2,4]
longitude_2 = coffee_shop_location.iloc[2,5]
search_query = 'Coffee'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_2, longitude_2, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results
venues = results['response']['venues']
Starbucks_1 = json_normalize(venues)
Starbucks_1.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{u'pluralName': u'Cafés', u'primary': True, u...",False,5c86b682da2e00002cf95781,"600 University Avenue, Room #202",CA,Toronto,Canada,NaN,155,"[600 University Avenue, Room #202, Toronto ON ...","[{u'lat': 43.657473, u'lng': -79.390637, u'lab...",43.657473,-79.390637,M5G 1X5,ON,TEMPORARILY CLOSED-Second Cup Coffee Co. featu...,v-1594035354
1,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,4dbae18d4df044e524bbb9de,200 Elizabeth St,CA,Toronto,Canada,Toronto General Hospital - Gerrard/1st Floor,128,[200 Elizabeth St (Toronto General Hospital - ...,"[{u'lat': 43.6584756, u'lng': -79.3874523, u'l...",43.658476,-79.387452,M5G 2C4,ON,Tim Hortons,v-1594035354


In [92]:
Starbucks_1.shape

(2, 17)

There are 2 venues relating coffee nearby Starbucks shop 1

# 4. Test for area nearby Tim Hortons shop 1

In [94]:
latitude_3 = coffee_shop_location.iloc[3,4]
longitude_3 = coffee_shop_location.iloc[3,5]
search_query = 'Coffee'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_3, longitude_3, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results
venues = results['response']['venues']
Tim_Hortons_1 = json_normalize(venues)
Tim_Hortons_1.head()

,categories,hasPerk,id,location.cc,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,name,referralId
0,[],False,4d8bbf07342b3704a6f3865c,CA,Canada,159,[Canada],"[{u'lat': 43.661659, u'lng': -79.395588, u'lab...",43.661659,-79.395588,MIE Coffee House,v-1594035514


In [95]:
Tim_Hortons_1.shape

(1, 12)

There is only one venue relating coffee nearby Tim Hortons shop 1

# 5. Test for area nearby Starbucks shop 2

In [96]:
latitude_4 = coffee_shop_location.iloc[4,4]
longitude_4 = coffee_shop_location.iloc[4,5]
search_query = 'Coffee'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_4, longitude_4, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results
venues = results['response']['venues']
Starbucks_2 = json_normalize(venues)
Starbucks_2.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{u'pluralName': u'Cafés', u'primary': True, u...",False,5c86b6b9cb3fd2002cd9c9a2,179 College Street,CA,Toronto,Canada,227,"[179 College Street, Toronto ON M5T 1P7, Canada]","[{u'lat': 43.6588724404, u'lng': -79.394158290...",43.658872,-79.394158,M5T 1P7,ON,Second Cup Coffee Co. featuring Pinkberry Froz...,v-1594041013
1,[],False,4d8bbf07342b3704a6f3865c,NaN,CA,NaN,Canada,173,[Canada],"[{u'lat': 43.661659, u'lng': -79.395588, u'lab...",43.661659,-79.395588,NaN,NaN,MIE Coffee House,v-1594041013


In [97]:
Starbucks_2.shape

(2, 16)

There are 2 venues relating coffee nearby Starbucks shop 2.

# 6. Test for area nearby Tim Hortons shop 2

In [98]:
latitude_5 = coffee_shop_location.iloc[5,4]
longitude_5 = coffee_shop_location.iloc[5,5]
search_query = 'Coffee'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_5, longitude_5, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results
venues = results['response']['venues']
Tim_Hortons_2 = json_normalize(venues)
Tim_Hortons_2.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{u'pluralName': u'Cafés', u'primary': True, u...",False,5c86b682da2e00002cf95781,"600 University Avenue, Room #202",CA,Toronto,Canada,221,"[600 University Avenue, Room #202, Toronto ON ...","[{u'lat': 43.657473, u'lng': -79.390637, u'lab...",43.657473,-79.390637,M5G 1X5,ON,TEMPORARILY CLOSED-Second Cup Coffee Co. featu...,v-1594041209


In [100]:
Tim_Hortons_2.shape

(1, 16)

There is only 1 venue relating coffee nearby Tim Hortons shop 2

# 7. Test for area nearby Tim Hortons shop 3

In [102]:
latitude_6 = coffee_shop_location.iloc[6,4]
longitude_6 = coffee_shop_location.iloc[6,5]
search_query = 'Coffee'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_6, longitude_6, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results
venues = results['response']['venues']
Tim_Hortons_3 = json_normalize(venues)
Tim_Hortons_3.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{u'pluralName': u'Cafés', u'primary': True, u...",False,5c86b682da2e00002cf95781,"600 University Avenue, Room #202",CA,Toronto,Canada,78,"[600 University Avenue, Room #202, Toronto ON ...","[{u'lat': 43.657473, u'lng': -79.390637, u'lab...",43.657473,-79.390637,M5G 1X5,ON,TEMPORARILY CLOSED-Second Cup Coffee Co. featu...,v-1594041459


In [101]:
Tim_Hortons_3.shape

(1, 16)

There is only 1 venue relating coffee nearby Tim Hortons shop 3

# 8. Test for area nearby Tim Hortons shop 4

In [103]:
latitude_7 = coffee_shop_location.iloc[7,4]
longitude_7 = coffee_shop_location.iloc[7,5]
search_query = 'Coffee'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_7, longitude_7, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results
venues = results['response']['venues']
Tim_Hortons_4 = json_normalize(venues)
Tim_Hortons_4.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{u'pluralName': u'Coffee Shops', u'primary': ...",False,4dbae18d4df044e524bbb9de,200 Elizabeth St,CA,Toronto,Canada,Toronto General Hospital - Gerrard/1st Floor,111,[200 Elizabeth St (Toronto General Hospital - ...,"[{u'lat': 43.6584756, u'lng': -79.3874523, u'l...",43.658476,-79.387452,M5G 2C4,ON,Tim Hortons,v-1594041513


In [104]:
Tim_Hortons_4.shape

(1, 17)

There is only 1 venue relating coffee nearby Tim Hortons shop 4.

To conclude, the chosen location is nearby the Neo Coffee Bar, which is located at the following coordinations:

In [105]:
print(latitude,longitude)

(43.66014, -79.38587)
